In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [2]:
key_path = key_path
project = project_id

# Sources
currency_table = 'silver_currency_data'
currency_dataset = 'silver'
indicators_table = 'silver_indicators'
indicators_dataset = 'silver'
ticker_info_table = 'silver_ticker_info'
ticker_info_dataset = 'silver'
macro_data_table = 'silver_fred_macro_data'
macro_data_dataset = 'silver'
cluster_table = 'gold_clustering_sp500'
cluster_dataset = 'gold'

# Tables id
table_conca_currency = f'{project}.{currency_dataset}.{currency_table}'
table_conca_indicators = f'{project}.{indicators_dataset}.{indicators_table}'
table_conca_ticker_info = f'{project}.{ticker_info_dataset}.{ticker_info_table}'
table_conca_macro_data = f'{project}.{macro_data_dataset}.{macro_data_table}'
table_conca_cluster = f'{project}.{cluster_dataset}.{cluster_table}'

# Loading tables
table_to_save = 'gold_main_sp500'
dataset_to_save = 'gold'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [10]:
df = bigquery.run_query(
    f"""
    SELECT
        indi.*,
        cl.cluster,
        info.sector_group,
        info.industry_group,
        cu.USDEUR,
        cu.USDJPY,
        cu.USDGBP,
        cu.USDCHF,
        cu.USDCNY,
        mc.BOPGSTB,
        mc.CPIAUCSL,
        mc.FEDFUNDS,
        mc.GDP,
        mc.PPIACO,
        mc.RSAFS,
        mc.UMCSENT,
        mc.UNRATE
    FROM {table_conca_indicators} AS indi
    INNER JOIN {table_conca_ticker_info} AS info
        ON info.ticker = indi.ticker    
    INNER JOIN {table_conca_currency} AS cu
        ON indi.date = cu.Date
    INNER JOIN {table_conca_macro_data} AS mc
        ON indi.date = mc.Date
    INNER JOIN {table_conca_cluster} AS cl
        ON indi.ticker = cl.ticker        
    """
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183479 entries, 0 to 1183478
Data columns (total 48 columns):
 #   Column             Non-Null Count    Dtype              
---  ------             --------------    -----              
 0   date               1183479 non-null  datetime64[us, UTC]
 1   open               1183479 non-null  float64            
 2   high               1183479 non-null  float64            
 3   low                1183479 non-null  float64            
 4   close              1183479 non-null  float64            
 5   adjclose           1183479 non-null  float64            
 6   volume             1183479 non-null  Int64              
 7   ticker             1183479 non-null  object             
 8   id                 1183479 non-null  object             
 9   SMA_20             1183479 non-null  float64            
 10  EMA_50             1183479 non-null  float64            
 11  WILLR              1183479 non-null  float64            
 12  ATR           

In [11]:
# Calculate the number of null values per column
nulls_by_column = df.isnull().sum()

# Sort the result in descending order
nulls_by_column_sorted = nulls_by_column.sort_values(ascending=False)

nulls_by_column_sorted

date                 0
open                 0
daily_return         0
macd                 0
macd_signal          0
macd_hist            0
stoch                0
stoch_d              0
cluster              0
industry             0
sector               0
USDEUR               0
USDJPY               0
USDGBP               0
USDCHF               0
USDCNY               0
BOPGSTB              0
CPIAUCSL             0
FEDFUNDS             0
GDP                  0
PPIACO               0
RSAFS                0
UMCSENT              0
bb_bbl               0
bb_bbm               0
bb_bbh               0
WILLR                0
high                 0
low                  0
close                0
adjclose             0
volume               0
ticker               0
id                   0
SMA_20               0
EMA_50               0
ATR                  0
CDL_LONGLINE         0
ADX                  0
CCI                  0
OBV                  0
RSI                  0
CDL_DOJI             0
CDL_HAMMER 

In [12]:
# Convertimos las variables categoricas en dummies
columns_to_encode = ['sector_group', 'industry_group']
categorical_df = pd.DataFrame([], columns=[])

# Aplicamos el one hot encoding
for col in columns_to_encode:
  new_cols_oneHot = pd.get_dummies(df[col], prefix=col)*1.0 # multiplico por 1.0 para convertir True/False a números directamente
  categorical_df = pd.concat([categorical_df, new_cols_oneHot], axis=1)

# Unimos en el mismo df
df_final = pd.concat([df, categorical_df], axis=1, join='inner')

# Eliminamos las variables categoricas que ya hemos transformado
df_final = df_final.drop(columns=columns_to_encode)

df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183479 entries, 0 to 1183478
Columns: 169 entries, date to industry_Waste Management
dtypes: Int64(8), datetime64[us, UTC](1), float64(158), object(2)
memory usage: 1.5+ GB


In [18]:
# import pandas as pd
# 
# def count_unique_values(df):
#     """
#     Counts the number of unique values in each column of the DataFrame
#     and sorts the results in descending order.
# 
#     Parameters:
#     df (pd.DataFrame): The DataFrame for which to count unique values.
# 
#     Returns:
#     pd.Series: A series where the index is column names and the values are 
#                the count of unique values, sorted in descending order.
#     """
#     # Count unique values for each column
#     unique_counts = df.nunique()
#     
#     # Sort the results in descending order
#     sorted_unique_counts = unique_counts.sort_values(ascending=False)
#     
#     return sorted_unique_counts
# 
# count_df = count_unique_values(df_final)
# count_df


macd_signal                               1183479
EMA_50                                    1183479
macd                                      1183479
macd_hist                                 1183479
ADX                                       1183479
                                           ...   
industry_Communication_Equipment                2
industry_Chemicals                              2
industry_Capital_Markets                        2
industry_Building_Products___Equipment          2
industry_Waste_Management                       2
Length: 169, dtype: int64

In [16]:
import pandas as pd
import re

# Define a function to clean column names
def clean_column_names(df):
    # Replace invalid characters with an underscore
    df.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', col) for col in df.columns]
    # Ensure column names do not exceed 300 characters
    df.columns = [col[:300] for col in df.columns]
    return df

# Assume 'df' is your DataFrame
df_cleaned = clean_column_names(df_final)

In [17]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df_cleaned, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
